For this assignment, you have 4 files containing information about persons.

The files are:
* `personal_info.csv` -   personal information such as name, gender, etc. (one row per person)
* `vehicles.csv` -   what vehicle people own (one row per person)
* `employment.csv` -   where a person is employed (one row per person)
* `update_status.csv` -   when the person's data was created and last updated

Each file contains a key, `SSN`, which **uniquely** identifies a person.

This key is present in **all** four files.

You are guaranteed that the same SSN value is present in **every** file, and that it only appears **once per file**.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

##### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

##### Goal 2

Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

##### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

##### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [239]:
import csv

In [240]:
personal_info = r'C:\Users\anila\Desktop\AI\EPAI-Phase1\S16_Context_Managers\personal_info.csv'
employment = r'C:\Users\anila\Desktop\AI\EPAI-Phase1\S16_Context_Managers\employment.csv'
update_status = r'C:\Users\anila\Desktop\AI\EPAI-Phase1\S16_Context_Managers\update_status.csv'
vehicles = r'C:\Users\anila\Desktop\AI\EPAI-Phase1\S16_Context_Managers\vehicles.csv'

# Goal1

**Personal_Info check**

In [420]:
with open(personal_info) as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        if i <= 2:
            print(vals)
    print(i)

ssn,first_name,last_name,gender,language

100-53-9824,Sebastiano,Tester,Male,Icelandic

101-71-4702,Cayla,MacDonagh,Female,Lao

1001


In [421]:
personal_info_type = ['INT', 'STRING', 'STRING', 'STRING','STRING']

**Vehicles check**

In [422]:
with open(vehicles) as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        if i <= 2:
            print(vals)
    print(i)
    

ssn,vehicle_make,vehicle_model,model_year

100-53-9824,Oldsmobile,Bravada,1993

101-71-4702,Ford,Mustang,1997

1001


In [423]:
vehicles_type= ['INT','STRING','STRING','INT']

**Employment Check**

In [424]:
with open(employment) as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        if i <= 2:
            print(vals)
    print(i)

employer,department,employee_id,ssn

Abbott, Bahringer and Goyette,Marketing,41-5601892,283-04-6835

Abshire Inc,Research and Development,55-8987515,868-32-1805

1001


In [425]:
employment_type = ['STRING','STRING','INT','INT']  # But we need to combine items like these -> Abbott, Bahringer and Goyette

**Update_Status Check**

In [426]:
with open(update_status) as file:
    file_vals=iter(file)
    for i,vals in enumerate(file_vals):
        if i <= 2:
            print(vals)
    print(i)

ssn,last_updated,created

100-53-9824,2017-10-07T00:14:42Z,2016-01-24T21:19:30Z

101-71-4702,2017-01-23T11:23:17Z,2016-01-27T04:32:57Z

1001


In [427]:
status_type = ['INT','DATE','DATE']

**Casting Function**

In [428]:
from datetime import datetime
def type_casting(col_data,input_val):
    if col_data == 'DATE':
        return datetime.strptime(input_val, '%Y-%m-%dT%H:%M:%SZ').date()
    elif col_data == 'INT':
        return int(input_val.replace('-','_'))
    else:
        return input_val

In [429]:
def cast_each_row(col_data_type,row_val):
    return [type_casting(col_data, value) for col_data, value in zip(col_data_type, row_val)]

In [430]:
from collections import namedtuple
import csv
from itertools import islice

def gen_fun(input_file,col_data_type, nt_name):
    with open(input_file) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        headers = next(iter(rows))
        named_tuple = namedtuple(nt_name, headers)
        for row in rows:
            if len(row) > 4 and nt_name == 'employment':
                row_new = []
                row_new.append(str(row[0] + row[1]))
                row_new.extend(row[2:])
                row = row_new
            yield named_tuple(*cast_each_row(col_data_type, row))

In [431]:
personal_data = gen_fun(personal_info, personal_info_type, 'personal_info')
for data in islice(personal_data, 3):
    print(data)
    personal_fields = data._fields

personal_info(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
personal_info(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
personal_info(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [432]:
vehicles_data = gen_fun(vehicles,vehicles_type, 'vehicles')
for data in islice(vehicles_data, 3):
    print(data)
    vehicles_fields = data._fields

vehicles(ssn=100539824, vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
vehicles(ssn=101714702, vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
vehicles(ssn=101840356, vehicle_make='GMC', vehicle_model='Yukon', model_year=2005)


In [433]:
status_data = gen_fun(update_status,status_type, 'update_status')
for data in islice(status_data, 3):
    print(data)
    status_fields = data._fields

update_status(ssn=100539824, last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24))
update_status(ssn=101714702, last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27))
update_status(ssn=101840356, last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21))


In [434]:
employment_data = gen_fun(employment, employment_type, 'employment')
for data in islice(employment_data, 3):
    print(data)
    employment_fields = data._fields

employment(employer='Abbott Bahringer and Goyette', department='Marketing', employee_id=415601892, ssn=283046835)
employment(employer='Abshire Inc', department='Research and Development', employee_id=558987515, ssn=868321805)
employment(employer='Abshire-Hahn', department='Marketing', employee_id=940531814, ssn=667877090)


# Goal 2

In [485]:
print(personal_fields)
print(vehicles_fields)
print(employment_fields)
print(status_fields)

('ssn', 'first_name', 'last_name', 'gender', 'language')
('ssn', 'vehicle_make', 'vehicle_model', 'model_year')
('employer', 'department', 'employee_id', 'ssn')
('ssn', 'last_updated', 'created')


In [486]:
combined_header = personal_fields + vehicles_fields[1:] + employment_fields[:-1] + status_fields[1:]
combined_header

('ssn',
 'first_name',
 'last_name',
 'gender',
 'language',
 'vehicle_make',
 'vehicle_model',
 'model_year',
 'employer',
 'department',
 'employee_id',
 'last_updated',
 'created')

In [490]:
personal_data = gen_fun(personal_info, personal_info_type, 'personal_info')
vehicles_data = gen_fun(vehicles,vehicles_type, 'vehicles')
status_data = gen_fun(update_status,status_type, 'update_status')
employment_data = gen_fun(employment, employment_type, 'employment')

In [491]:
def ssn_match():
    for pers, vehi, emp, status in zip(personal_data, vehicles_data, employment_data, status_data):
        ssn_match_info = namedtuple("ssn_match_info", combined_header)
        combined_data  = [*pers] + [*vehi][1:] + [*emp][:-1] + [*status][1:]
        yield ssn_match_info(*combined_data)

In [492]:
ssn_matched=ssn_match()
for data in islice(ssn_matched, 1000):
    print(data)

ssn_match_info(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993, employer='Abbott Bahringer and Goyette', department='Marketing', employee_id=415601892, last_updated=datetime.date(2017, 10, 7), created=datetime.date(2016, 1, 24))
ssn_match_info(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', vehicle_make='Ford', vehicle_model='Mustang', model_year=1997, employer='Abshire Inc', department='Research and Development', employee_id=558987515, last_updated=datetime.date(2017, 1, 23), created=datetime.date(2016, 1, 27))
ssn_match_info(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005, employer='Abshire-Hahn', department='Marketing', employee_id=940531814, last_updated=datetime.date(2017, 10, 4), created=datetime.date(2016, 9, 21))
ssn_m

ssn_match_info(ssn=534506091, first_name='Florencia', last_name='Abramof', gender='Female', language='Arabic', vehicle_make='Chevrolet', vehicle_model='Silverado', model_year=2007, employer='McCullough-Durgan', department='Research and Development', employee_id=9482594, last_updated=datetime.date(2017, 3, 27), created=datetime.date(2016, 11, 19))
ssn_match_info(ssn=534680554, first_name='Ira', last_name='Swanne', gender='Female', language='Chinese', vehicle_make='Infiniti', vehicle_model='G37', model_year=2011, employer='McDermott Inc', department='Research and Development', employee_id=907715733, last_updated=datetime.date(2018, 1, 27), created=datetime.date(2016, 1, 24))
ssn_match_info(ssn=534868290, first_name='Ester', last_name='Jepson', gender='Female', language='Albanian', vehicle_make='Jeep', vehicle_model='Grand Cherokee', model_year=2000, employer='McDermott-Hettinger', department='Marketing', employee_id=895009277, last_updated=datetime.date(2017, 2, 13), created=datetime.dat

# Goal 3

In [536]:
personal_data = gen_fun(personal_info, personal_info_type, 'personal_info')
vehicles_data = gen_fun(vehicles,vehicles_type, 'vehicles')
status_data = gen_fun(update_status,status_type, 'update_status')
employment_data = gen_fun(employment, employment_type, 'employment')

In [537]:
from itertools import islice
ssn_matched=ssn_match()

def stale_records(input_date):
    date = datetime.strptime(input_date, '%d-%m-%Y').date()
    for data in islice(ssn_matched, 1000):
        if data.last_updated < date:
            yield data
        if data.ssn == 899476116:
            print('End of records')

In [538]:
input_date = '03-01-2017'
stale_record=stale_records(input_date)

In [539]:
for data in stale_record:
    print(data.ssn, data.first_name, data.last_updated)

390743062 Cara 2017-01-01
644708292 Glyn 2017-01-02
669129323 Peter 2017-01-02
End of records


# Goal 4

In [481]:
personal_data = gen_fun(personal_info, personal_info_type, 'personal_info')
vehicles_data = gen_fun(vehicles,vehicles_type, 'vehicles')
status_data = gen_fun(update_status,status_type, 'update_status')
employment_data = gen_fun(employment, employment_type, 'employment')
all_record = ssn_match()

make_count_male = dict()
make_count_female = dict()
for _ in islice(all_record,1000):
    if _.gender == 'Male':
        make_count_male[_.vehicle_make] = make_count_male[_.vehicle_make] + 1 if _.vehicle_make in make_count_male else 1
    else:
        make_count_female[_.vehicle_make] = make_count_female[_.vehicle_make] + 1 if _.vehicle_make in make_count_female else 1
        

In [482]:
max_value = max(list(make_count_male.values()))
print([(k, v) for k, v in make_count_male.items() if v == max_value])

[('Ford', 44)]


In [483]:
max_value = max(list(make_count_female.values()))
print([(k, v) for k, v in make_count_female.items() if v == max_value])

[('Ford', 48), ('Chevrolet', 48)]
